In [4]:
!pip install seaborn



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:

%pip install matplotlib
%pip install seaborn
%pip install pandas numpy scikit-learn

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
def balance_check(data, target):
    try:
        df = pd.read_csv(data)
    except:
        print("Error Reading cannot be found")
        return

    #now check value counts
    ones_percentage = df[target].value_counts(normalize=True).loc[1] * 100
    print(f"Percentage of 1s for : {ones_percentage:.2f}%")

    return



In [15]:
balance_check("data/train.csv/train.csv","toxic")
balance_check("data/train.csv/train.csv","severe_toxic")
balance_check("data/train.csv/train.csv","obscene")
balance_check("data/train.csv/train.csv","threat")
balance_check("data/train.csv/train.csv","insult")
balance_check("data/train.csv/train.csv","identity_hate")

Percentage of 1s: 9.58%
Percentage of 1s: 1.00%
Percentage of 1s: 5.29%
Percentage of 1s: 0.30%
Percentage of 1s: 4.94%
Percentage of 1s: 0.88%


In [ ]:
df1 = pd.read_csv("data/train.csv/train.csv")
df1.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:

%pip install nltk

%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

%pip install transformers


  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.2 MB/s eta 0:00:00
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.6/284.6 kB 26.5 MB/s eta 0:00:00
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/nightly/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 59.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertModel
from tqdm import tqdm


nltk.download('punkt')
nltk.download('stopwords')

def load_dataset(file_path):
    try:
        df = pd.read_csv(file_path)
        
        print("Dataset loaded successfully!")
        print(f"Shape: {df.shape}")
        
        # check for null values

        print("\nNull values per column:")
        print(df.isnull().sum())
        
        return df
    
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

def clean_text(text):
    if not isinstance(text, str):
        return ""
    
    # remove artifacts and wikimarkups




    text = re.sub(r'==.*?==+', '', text)
    text = re.sub(r'\[\[.*?\]\]', '', text)
    text = re.sub(r'\{\{.*?\}\}', '', text)
    text = re.sub(r'__[A-Z]+__', '', text)
    text = re.sub(r'&[a-z]+;', '', text)
    text = re.sub(r'<.*?>', '', text)
    



    # URLs, IPs, timestamps, extra spaces, and lowercasing

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b', '', text)
    text = re.sub(r'\d{1,2}:\d{2},?\s*\d{1,2}\s+\w+\s+\d{4}', '', text)
    text = re.sub(r"[^\w\s'!?.]", ' ', text)
    text = re.sub(r'\s+', ' ', text).strip().lower()
    
    return text

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    



    # stopwords list, we can always change this 

    custom_stopwords = {
        'wikipedia', 'page', 'article', 'edit', 'user', 'talk',
        'section', 'redirect', 'category', 'file', 'image', 'template'
    }
    stop_words.update(custom_stopwords)
    



    # highlight important negation words of the sentence

    negation_words = {
        'not', 'no', 'never', 'none', 'nor', 'neither',
        "don't", "isn't", "aren't", "wasn't", "weren't",
        "haven't", "hasn't", "hadn't", "won't", "wouldn't"
    }
    stop_words -= negation_words
    


    # tokenize and filter the text and words

    tokens = word_tokenize(text)
    filtered_tokens = [
        word for word in tokens 
        if word not in stop_words and len(word) > 2 and not word.startswith("'")
    ]
    
    return ' '.join(filtered_tokens)




def initialize_bert():
    print("\nInitializing BERT model...")
    try:
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertModel.from_pretrained('bert-base-uncased')
        print("BERT initialized successfully")
        return tokenizer, model
    except Exception as e:
        print(f"Error initializing BERT: {e}")
        return None, None

if __name__ == "__main__":
    FILE_PATH = "data/train.csv/train.csv"
    
    print("Starting Toxic Comment Classification Pipeline")
    
    print("\n=== PHASE 1: DATA LOADING ===")
    df = load_dataset(FILE_PATH)
    
    if df is not None:
        print("\n=== PHASE 2: TEXT CLEANING ===")
        df['cleaned_text'] = df['comment_text'].apply(clean_text)
        print("\n=== PHASE 3: STOPWORD REMOVAL ===")
        df['processed_text'] = df['cleaned_text'].apply(remove_stopwords)
        


        # Show examples
        print("\nText processing examples:")
        for i in range(3):
            print(f"\nOriginal {i}:")
            print(df['comment_text'].iloc[i])
            print("\nProcessed:")
            print(df['processed_text'].iloc[i])
            print("-"*50)




        print("\n=== PHASE 4: BERT INITIALIZATION ===")
        bert_tokenizer, bert_model = initialize_bert()
        
        if bert_tokenizer:
            print("\nTokenization example:")
            sample_text = df['processed_text'].iloc[0]
            print(f"\nSample text: {sample_text}")
            
            tokens = bert_tokenizer.tokenize(sample_text)
            print(f"\nTokens: {tokens}")
            
            token_ids = bert_tokenizer.encode(sample_text, add_special_tokens=True)
            print(f"\nToken IDs: {token_ids}")

    print("\nPipeline execution complete")

[nltk_data] Downloading package punkt to /Users/tyran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/tyran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Starting Toxic Comment Classification Pipeline

=== PHASE 1: DATA LOADING ===
Dataset loaded successfully!
Shape: (159571, 8)

Null values per column:
id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

=== PHASE 2: TEXT CLEANING ===

=== PHASE 3: STOPWORD REMOVAL ===

Text processing examples:

Original 0:
Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27

Processed:
explanation edits made username hardcore metallica fan reverted n't vandalisms closure gas voted new york dolls fac please n't remove since retired
--------------------------------------------------

Original 1:
D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51